In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from typing import Union
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, DoubleType, IntegerType, TimestampType, StringType

In [4]:
MAX_MEMORY="7g"
spark = SparkSession.builder.appName("covid18")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

22/08/25 15:32:20 WARN Utils: Your hostname, hyunj resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface eno1)
22/08/25 15:32:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 15:32:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/25 15:32:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [16]:
def getCovid19Info(start_date: date, end_date: date):
    url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
    api_key_utf8 = 'Ax%2FnB%2FW10KaDGhIK%2Fds3YS6Jx7V%2BkUR%2FBs8l0L%2Ff6icPTFibZkV1lkgsTvlLWFhI5MhUeRPbqYm5uuFXEovnBA%3D%3D'
    api_key_decode = requests.utils.unquote(api_key_utf8, encoding='utf-8')

    params ={
        'serviceKey' : api_key_decode,
        'startCreateDt' : int('{:04d}{:02d}{:02d}'.format(start_date.year, start_date.month, start_date.day)),
        'endCreateDt' : int('{:04d}{:02d}{:02d}'.format(end_date.year, end_date.month, end_date.day)),
    }

    response = requests.get(url, params=params)
    content = response.text
    elapsed_us = response.elapsed.microseconds
    print('Reqeust Done, Elapsed: {} seconds'.format(elapsed_us / 1e6))#100만
    
    return BeautifulSoup(content, "lxml")#불러온 text 데이터를 python xml인 lxml로 변환

def getCovid19SparkDataFrame(start_date: Union[date, datetime], end_date: Union[date,datetime]):
    #key값의 데이터 타입 정의
    convert_method = {
        'accdefrate' : float,
        'accexamcnt' : int,
        'createdt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
        'deathcnt' : int,
        'decidecnt' : int,
        'seq' : int,
        'statedt' : str,
        'statetime' : str,
        'updatedt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
    }
    temp = getCovid19Info(start_date, end_date)
    items = temp.find('items')#item만 추출
    item_list = []
    for item in items:
        item_dict = {}
        for tag in list(item):
            try:
                item_dict[tag.name] = convert_method[tag.name](tag.text)
            except Exception:
                item_dict[tag.name] = None
        item_list.append(item_dict)#lxml데이터를 dictionary in list로 변환
    
    CovidInfo_item_Schema = StructType([
        StructField('accdefrate', DoubleType(), True),#누적확진률
        StructField('accexamcnt', IntegerType(), True),#누적의심신고검사자
        StructField('createdt', TimestampType(),True),#등록일시분초
        StructField('deathcnt', IntegerType(), True),#사망자 수
        StructField('decidecnt', IntegerType(), True),#확진자 수
        StructField('seq', IntegerType(), True),#게시글 번호(감염현황 고유값)
        StructField('statedt', StringType(), True),#기준일
        StructField('statetime', StringType(), True),#기준시간
        StructField('updatedt', TimestampType(), True)#수정일시분초
    ])
    
    df_Covid19 = spark.createDataFrame(item_list,CovidInfo_item_Schema)
    return df_Covid19

In [19]:
getCovid19SparkDataFrame(20220707,20220707)

AttributeError: 'int' object has no attribute 'year'

In [14]:
def getAllCovid19Data():
    df_Covid19 = getCovid19SparkDataFrame(date(2019,1,1), now)
    print("Today: {}\nLoaded {} Records".format(now.strftime('%Y-%m-%d'),df_Covid19.distinct().count()))
    return df_Covid19

In [10]:
def getCovid19SparkDataFrame(start_date: Union[date, datetime], end_date: Union[date,datetime]):
    #key값의 데이터 타입 정의
    convert_method = {
        'accdefrate' : float,
        'accexamcnt' : int,
        'createdt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
        'deathcnt' : int,
        'decidecnt' : int,
        'seq' : int,
        'statedt' : str,
        'statetime' : str,
        'updatedt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
    }

In [11]:
temp = getCovid19Info(start_date, end_date)
items = temp.find('items')#item만 추출

NameError: name 'start_date' is not defined